In [12]:
import sys
sys.path.append("../../")

import django
import pylab as pl

from django.db import connections
from firecares.firestation.models import FireDepartment, FireStation
from firecares.utils import dictfetchall
from pandas import DataFrame

%matplotlib inline

django.setup()

print "Number of unmatched fire stations:", FireStation.objects.filter(department__isnull=True).count()

cursor = connections['default'].cursor()

queryUnmatchedStations = """
select a.name as firestation_name, b.usgsstructuredata_ptr_id as firestation_id, b.department_id as dep_id, ST_ASTEXT(ST_TRANSFORM(d.geom,900913)) as firestation_location
from firestation_usgsstructuredata a
inner join firestation_firestation b
  on a.id=b.usgsstructuredata_ptr_id
join firecares_core_address d 
  on b.station_address_id=d.id
where b.department_id is null and d.geom is not null
"""
#Query fireDepartments within 50 miles of station
queryNearbyDepartments = """
select c.name as department_name, c.id as dep_id, 
ST_DISTANCE(ST_TRANSFORM(e.geom,900913),ST_GEOMFROMTEXT(%s,900913)) * 0.000621371 as dis_miles,
levenshtein(c.name,%s) as dis_name
from firestation_firedepartment c 
inner join firecares_core_address e 
  on c.headquarters_address_id=e.id
where e.geom is not null and
ST_DISTANCE(ST_TRANSFORM(e.geom,900913),ST_GEOMFROMTEXT(%s,900913)) * 0.000621371 <= 50 and
levenshtein(c.name,%s) <= 50; 
"""

queryUpdateFireStation = """
update firestation_firestation 
set department_id = %s
where firestation_firestation.usgsstructuredata_ptr_id = %s
"""

cursor.execute(queryUnmatchedStations)
fireStations = dictfetchall(cursor)
print cursor.rowcount

matchedDict = dict()
for fireStation in fireStations:
    params = [fireStation['firestation_location'],fireStation['firestation_name'],fireStation['firestation_location'],fireStation['firestation_name']]
    reults = cursor.execute(queryNearbyDepartments,params)
    nearbyDepartments = dictfetchall(cursor)
    smallestDistanceMiles = 100
    smallestDistanceName = 100
    closestDepID = 0
    closestDepName = ''
    if cursor.rowcount == 0:
        continue
    for fireDepartment in nearbyDepartments:
        if fireDepartment['dis_miles'] < smallestDistanceMiles and fireDepartment['dis_name'] < smallestDistanceName:
            smallestDistanceMiles = fireDepartment['dis_miles']
            smallestDistanceName = fireDepartment['dis_name']
            closestDepID = fireDepartment['dep_id']
            closestDepName = fireDepartment['department_name']
    fireStation['dep_id'] = closestDepID
    if matchedDict.get(closestDepName) == None:
        matchedDict[closestDepName] = 1
    else:
        matchedDict[closestDepName] = matchedDict[closestDepName] + 1

print "Done Matching"

for departmentName,numStations in matchedDict.iteritems():
    print departmentName, numStations
     

Number of unmatched fire stations: 50723
50331
Done Matching
Niagara Fire Company 1
Laurel Hill Volunteer Fire Company 2
Forks Fire District No.3 2
South Brooks Volunteer Fire Department 1
South Lee Fire Department 2
New Baltimore Fire District 2
Millbury Fire Department 7
Hammondville Volunteer Fire Department 3
Bakersfield Fire Department 27
Rockville Centre Fire Department 5
Downs Community Fire Protection District 1
Dunstable Fire Department 1
Colestin Rural Fire Protection District 7
Agency Volunteer Fire & Rescue 1
Southern Webster County Fire Protection District 4
Newmanstown Volunteer Fire Company 1
Maywood-Wellfleet Rural Fire District 1
Boca Grande Fire Department 13
Trafford Fire Company 1 1
Rush Volunteer Fire Department 5
Cankton Volunteer Fire Department 3
Beverly Volunteer Fire Department 3
Yelvington Fire Department 1
Savannah Fire & Emergency Services 11
Chambersburg Fire Department 1
Harleton Volunteer Fire Department 2
Central Emergency Services 6
North Union Volunte

In [11]:
cursor = connections['default'].cursor()

queryUnmatchedStations = """
select a.name as firestation_name, b.usgsstructuredata_ptr_id as firestation_id, b.department_id as dep_id, ST_ASTEXT(ST_TRANSFORM(d.geom,900913)) as firestation_location
from firestation_usgsstructuredata a
inner join firestation_firestation b
  on a.id=b.usgsstructuredata_ptr_id
join firecares_core_address d 
  on b.station_address_id=d.id
where b.department_id is not null and d.geom is not null
"""
#Query fireDepartments within 50 miles of station
queryNearbyDepartments = """
select c.name as department_name, c.id as dep_id, 
ST_DISTANCE(ST_TRANSFORM(e.geom,900913),ST_GEOMFROMTEXT(%s,900913)) * 0.000621371 as dis_miles,
levenshtein(c.name,%s) as dis_name
from firestation_firedepartment c 
inner join firecares_core_address e 
  on c.headquarters_address_id=e.id
where e.geom is not null and
ST_DISTANCE(ST_TRANSFORM(e.geom,900913),ST_GEOMFROMTEXT(%s,900913)) * 0.000621371 <= 50 and
levenshtein(c.name,%s) <= 50; 
"""

queryUpdateFireStation = """
update firestation_firestation 
set department_id = %s
where firestation_firestation.usgsstructuredata_ptr_id = %s
"""

cursor.execute(queryUnmatchedStations)
fireStations = dictfetchall(cursor)
print cursor.rowcount

matchedDict = dict()
for fireStation in fireStations:
    params = [fireStation['firestation_location'],fireStation['firestation_name'],fireStation['firestation_location'],fireStation['firestation_name']]
    reults = cursor.execute(queryNearbyDepartments,params)
    nearbyDepartments = dictfetchall(cursor)
    smallestDistanceMiles = 100
    smallestDistanceName = 100
    closestDepID = 0
    closestDepName = ''
    if cursor.rowcount == 0:
        continue
    for fireDepartment in nearbyDepartments:
        if fireDepartment['dis_miles'] < smallestDistanceMiles and fireDepartment['dis_name'] < smallestDistanceName:
            smallestDistanceMiles = fireDepartment['dis_miles']
            smallestDistanceName = fireDepartment['dis_name']
            closestDepID = fireDepartment['dep_id']
            closestDepName = fireDepartment['department_name']
    fireStation['dep_id'] = closestDepID
    if matchedDict.get(closestDepName) == None:
        matchedDict[closestDepName] = 1
    else:
        matchedDict[closestDepName] = matchedDict[closestDepName] + 1

print "Done Matching"

for departmentName,numStations in matchedDict.iteritems():
    print departmentName, numStations

1847
Done Matching
Manitou Springs Fire Department 2
Alta Vista Fire Department 1
Fairlawn Volunteer Fire Department  Inc. 2
Enumclaw Fire Department/ King County Fire District #28/ Rescue 1 2
Shavano Park Fire Department/ EMS 2
Delran Fire Department 3
Virginia Beach Fire Department 14
Beverly Hills Fire Department 28
Coral Gables Fire & Rescue Department 8
Austin Fire Department 43
Cimarron Hills Fire Department 6
Lake Worth Fire Department 1
Appalachia Volunteer Fire Department 1
Los Angeles County Fire Department 2
Buechel Fire Protection District 4
San Antonio Fire Department 24
Clinton Township Fire Department 4
Leon Valley Fire Department 8
Sacramento Metropolitan Fire District 6
Crystal Lake Fire Department 1
Estherville Fire Department 1
Boston Fire Department 29
Clovis Fire Department 6
Blackdog Fire Department 1
Edge Hill Fire Company 1 4
Gilbert Fire Department 8
Boeing Fire Protection - Mesa 5
Stanley Fire Department 1
Metro Nashville Airport Authority - Public Safety 1
Me